# Select reference samples
- 2020.12.29
- Idea：两两样本计算关联系数，填到一个三角阵中。方便后期调用。
- 尚未成功

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import dask.dataframe as dd

import function as func
import itertools
from datetime import datetime 

# Functions

In [34]:
def fetch_norm_rd(sampleID, sample_norm_file):
    df = pd.read_table(sample_norm_file,low_memory=False,header=None, sep='\t', \
                                 names=['chr', 'start','end','GC','RD',sampleID])
    return df[sampleID].to_frame()

# Processes

In [18]:
sample_norm_file='/home/rt2776/cnv_espresso/predict_pcgc/sample_norm_rd_gz_file.txt'

In [19]:
sample_norm_list = func.fileToList_tab(sample_norm_file)

In [61]:
func.showDateTime()
num = 0
combined_list = []
sampleID_list = []

for sample_rd_path, sampleID, in sample_norm_list:
    num += 1
    if num % 100 == 0:
        func.showDateTime('\t')
        print("Importing No.%d sample:%s from %s"%(num, sampleID, sample_rd_path)
        
    rd_df = fetch_norm_rd(sampleID, sample_rd_path)
    combined_list.append(rd_df.to_numpy())
    sampleID_list.append(sampleID)

combined_np_array = np.hstack(combined_list) # convert to nparray to accelerate the speed.
# combined_df = pd.DataFrame(combined_np_array)
# combined_df.columns = sampleID_list

func.showDateTime()

[2021.02.10-12:19:17]
[2021.02.10-12:19:36]	100 /home/rt2776/cnv_espresso/predict_pcgc/norm/1-00132.norm.cov.bed.norm.gz 1-00132
[2021.02.10-12:20:08]	200 /home/rt2776/cnv_espresso/predict_pcgc/norm/1-00229-02.norm.cov.bed.norm.gz 1-00229-02
[2021.02.10-12:21:02]	300 /home/rt2776/cnv_espresso/predict_pcgc/norm/1-00347-02.norm.cov.bed.norm.gz 1-00347-02
[2021.02.10-12:22:08]	400 /home/rt2776/cnv_espresso/predict_pcgc/norm/1-00448-01.norm.cov.bed.norm.gz 1-00448-01
[2021.02.10-12:23:07]	500 /home/rt2776/cnv_espresso/predict_pcgc/norm/1-00561-02.norm.cov.bed.norm.gz 1-00561-02
[2021.02.10-12:24:20]	600 /home/rt2776/cnv_espresso/predict_pcgc/norm/1-00679.norm.cov.bed.norm.gz 1-00679
[2021.02.10-12:25:15]	700 /home/rt2776/cnv_espresso/predict_pcgc/norm/1-00803.norm.cov.bed.norm.gz 1-00803
[2021.02.10-12:26:12]	800 /home/rt2776/cnv_espresso/predict_pcgc/norm/1-00893-02.norm.cov.bed.norm.gz 1-00893-02
[2021.02.10-12:26:55]	900 /home/rt2776/cnv_espresso/predict_pcgc/norm/1-01019-02.norm.cov.be

In [65]:
# faster way than `corrMatrix = combined_df.corr()`
# ref: https://stackoverflow.com/questions/56628363/computing-correlation-matrix-faster-in-pandas
## corrMatrix = pd.DataFrame(np.corrcoef(combined_df.values, rowvar=False), columns=combined_df.columns)

corrMatrix = pd.DataFrame(np.corrcoef(combined_np_array, rowvar=False), columns=sampleID_list)
func.showDateTime()

In [67]:
corrMatrix

,1-00002-03,1-00002,1-00018-01,1-00018,1-00025-01,1-00025-02,1-00025,1-00027,1-00030-01,1-00030-02,...,M007-22,M008-23,M008-24,M008-25,M009-26,M009-27,M009-28,M009-29,M009-30,M01-3
0,1.000000,0.220234,0.784167,0.317879,0.133597,0.286437,0.262733,0.359571,0.053178,0.381504,...,0.299478,0.248706,0.258330,0.318218,0.329520,0.328298,0.331854,0.325536,0.315801,0.044763
1,0.220234,1.000000,0.314669,0.251845,0.158529,0.364259,0.346921,0.742126,0.091256,0.657419,...,0.373872,0.337710,0.347205,0.392378,0.378033,0.378640,0.398070,0.410037,0.396740,0.132395
2,0.784167,0.314669,1.000000,0.443663,0.172813,0.366989,0.355530,0.476724,0.107546,0.492044,...,0.362153,0.330768,0.340007,0.389447,0.412321,0.413244,0.453534,0.444596,0.437372,0.092882
3,0.317879,0.251845,0.443663,1.000000,0.158866,0.353074,0.301435,0.409356,0.083293,0.441078,...,0.353907,0.318785,0.372494,0.376647,0.374293,0.403202,0.418186,0.374369,0.385075,0.049787
4,0.133597,0.158529,0.172813,0.158866,1.000000,0.687232,-0.085149,0.198144,0.022680,0.243933,...,0.558076,0.313007,0.719019,0.219167,0.314400,0.315032,0.320622,0.252566,0.312592,0.164418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5396,0.328298,0.378640,0.413244,0.403202,0.315032,0.793693,0.689822,0.540488,0.059366,0.661713,...,0.799370,0.719211,0.724645,0.886233,0.906896,1.000000,0.890090,0.890436,0.887601,0.320289
5397,0.331854,0.398070,0.453534,0.418186,0.320622,0.789842,0.734361,0.544736,0.063641,0.662538,...,0.795184,0.758502,0.749815,0.889987,0.905025,0.890090,1.000000,0.942901,0.904168,0.331618
5398,0.325536,0.410037,0.444596,0.374369,0.252566,0.766941,0.780328,0.554821,0.062747,0.655269,...,0.781138,0.765240,0.717437,0.910106,0.914959,0.890436,0.942901,1.000000,0.911856,0.342292
5399,0.315801,0.396740,0.437372,0.385075,0.312592,0.762797,0.780327,0.541987,0.060965,0.660342,...,0.763766,0.730812,0.718460,0.858055,0.868905,0.887601,0.904168,0.911856,1.000000,0.330733


In [ ]:
# sn.heatmap(corrMatrix[0:10,0:10], annot=True)
# plt.show()

In [87]:
sampleID_str = '\t'.join([sampleID for sampleID in sampleID_list])

In [88]:
corr_matrix_file = '/home/rt2776/cnv_espresso/predict_pcgc/corr_matrix.txt'
np.savetxt(corr_matrix_file,corrMatrix, delimiter="\t", header=sampleID_str)

# Select reference samples

In [89]:
cnv_info_file = '/home/rt2776/cnv_espresso/predict_pcgc/pcgc_NimbleGenV2_data.txt'

In [91]:
cnv_info_df = pd.read_table(cnv_info_file,low_memory=False, sep='\t')

In [96]:
cnv_info_df

,ID,Chr,Start,End,Band,CNV*,Syndrome_or_Gene,AnalysisObserved†,CardiacLesion(Diagnosis),ParentOrigin,Extracardiac,Genes_n,Size_kb,ref,batch
0,1-01401,1,59247993,59251097,p32.1,1,JUN,A,LVOT(HLHS),-,-,1,3.1,hg19,NimbleGenV2
1,1-03171,1,145586403,145799634,q21.1,3,"1q21.1,dup_or_GJA5‡","A,E",CTD(TOF_or_APVS),-,-,7,213.2,hg19,NimbleGenV2
2,1-01036,1,146631133,147416212,q21.1,3,"1q21.1,dup_or_GJA5‡",E,CTD(TOF),M,-,15,785.1,hg19,NimbleGenV2
3,1-01486,1,194201171,194304070,q24.2–q25,3,CDC73,A,LVOT(HLHS),-,Yes,0,102.9,hg19,NimbleGenV2
4,1-01536,2,70168995,70359345,p13.3,1,PCBP1,A,CTD(TOF_or_PA),-,-,5,190.4,hg19,NimbleGenV2
5,1-01401,2,102493466,103001458,q11.2–q12.1,1,MAP4K4,E,LVOT(HLHS),-,-,6,508.0,hg19,NimbleGenV2
6,1-01401,2,145155868,145274931,q22.3,1,Mowat-Wilson_or_ZEB2‡,E,LVOT(HLHS),-,-,1,119.1,hg19,NimbleGenV2
7,1-00762,3,60661,11712230,p26.1,3,"ARL8B,ARPC4,CAMK1,CAV3,CRBN,EMC3,ITPR1,SEC13,S...",A,ASD_or_PS(ASD),-,Yes,103,11651.6,hg19,NimbleGenV2
8,1-01049,3,15637812,15643461,p25.1,3,"BTD,HACL1",E,CTD(TOF),-,-,2,5.6,hg19,NimbleGenV2
9,1-01045,3,47780965,48309270,p21.31,3,"CDC25A,DHX30,MAP4,SMARCC1",A,LVOT(HLHS),-,-,14,528.3,hg19,NimbleGenV2


In [100]:
case_sampleID_list = []
for index, row in cnv_info_df.iterrows():  
        sampleID = row[0]
        case_sampleID_list.append(sampleID)
        
        sample_cnv_chr = row[1]
        sample_cnv_start = row[2]
        sample_cnv_end  = row[3]
        if row[5] == 1:
            sample_cnv_type = 'DEL'
        elif row[5] == 3:
            sample_cnv_type = 'DUP'
        else:
            print("Wrong type")
#         print(sampleID, sample_cnv_chr, sample_cnv_start, sample_cnv_end, sample_cnv_type)

In [103]:
case_sampleID_list = np.unique(case_sampleID_list)

In [152]:
for case_sampleID in case_sampleID_list:
    num_ref=100
    ref_sample_list = []

    print("case sampleID:",case_sampleID)
    ref_sample_df = corrMatrix[case_sampleID].sort_values(ascending=False)
    for i in range(1,num_ref+1):
        ref_sampleID = sampleID_list[ref_sample_df.index[i]]
        ref_sample_corr = ref_sample_df.iloc[i]
        ref_sample_list.append([ref_sampleID, ref_sample_corr])

    output_ref_file = '/home/rt2776/cnv_espresso/predict_pcgc/ref_samples/'+case_sampleID+'.ref.samples.txt'
    func.output_to_file(ref_sample_list,output_ref_file)

case sampleID: 1-00096
[INFO]: File outputs to /home/rt2776/cnv_espresso/predict_pcgc/ref_samples/1-00096.ref.samples.txt
[2021.02.10-14:13:54]
case sampleID: 1-00113
[INFO]: File outputs to /home/rt2776/cnv_espresso/predict_pcgc/ref_samples/1-00113.ref.samples.txt
[2021.02.10-14:13:54]
case sampleID: 1-00174
[INFO]: File outputs to /home/rt2776/cnv_espresso/predict_pcgc/ref_samples/1-00174.ref.samples.txt
[2021.02.10-14:13:54]
case sampleID: 1-00192
[INFO]: File outputs to /home/rt2776/cnv_espresso/predict_pcgc/ref_samples/1-00192.ref.samples.txt
[2021.02.10-14:13:54]
case sampleID: 1-00197
[INFO]: File outputs to /home/rt2776/cnv_espresso/predict_pcgc/ref_samples/1-00197.ref.samples.txt
[2021.02.10-14:13:54]
case sampleID: 1-00230
[INFO]: File outputs to /home/rt2776/cnv_espresso/predict_pcgc/ref_samples/1-00230.ref.samples.txt
[2021.02.10-14:13:54]
case sampleID: 1-00243
[INFO]: File outputs to /home/rt2776/cnv_espresso/predict_pcgc/ref_samples/1-00243.ref.samples.txt
[2021.02.10-14